In [106]:
import pandas as pd

In [107]:
df_basic_info = pd.read_csv("../data/processed/stocks-basic-info.csv")
df_prices = pd.read_csv("../data/processed/stocks-prices.csv", parse_dates=["DATE"])
df_fundaments = pd.read_csv("../data/processed/stocks-fundaments.csv", parse_dates=["DT_INI_EXERC", "DT_FIM_EXERC"])

In [108]:
df_basic_info.head()

,CD_CVM,DENOM_SOCIAL,CNPJ_CIA,SETOR_ATIV,CONTROLE_ACIONARIO,TICKERS,NUM_ORDINARIAS,NUM_PREFERENCIAIS,NUM_TOTAL
0,906,BANCO BRADESCO S.A.,60.746.948/0001-12,Bancos,PRIVADO,BBDC3;BBDC4,5330304681,5311865547,10642170228
1,1023,BANCO DO BRASIL S.A.,00.000.000/0001-91,Bancos,ESTATAL,BBAS11;BBAS12;BBAS3,5730834040,0,5730834040


In [109]:
print(df_prices.shape)
df_prices.head()

(4978, 4)


,CD_CVM,TICKER,DATE,PRICE
0,906,BBDC4,2014-10-06,8.793426
1,906,BBDC4,2014-10-07,8.735580
2,906,BBDC4,2014-10-08,8.661528
3,906,BBDC4,2014-10-09,8.830453
4,906,BBDC4,2014-10-10,8.388464


In [110]:
print(df_fundaments.shape)
df_fundaments.tail()

(20, 7)


,CD_CVM,DT_INI_EXERC,DT_FIM_EXERC,DS_CONTA,VL_CONTA,EXERC_YEAR,VL_CONTA_ROLLING_YEAR
15,1023,2023-10-01,2023-12-31,Lucro ou Prejuízo,7.940448e+09,2023,3.316559e+10
16,906,2024-01-01,2024-03-31,Lucro ou Prejuízo,4.202768e+09,2024,1.323172e+10
17,1023,2024-01-01,2024-03-31,Lucro ou Prejuízo,8.680714e+09,2024,3.455532e+10
18,906,2024-04-01,2024-06-30,Lucro ou Prejuízo,4.185407e+09,2024,1.334586e+10
19,1023,2024-04-01,2024-06-30,Lucro ou Prejuízo,9.485229e+09,2024,3.494804e+10


In [111]:
df_history = df_prices.merge(df_fundaments, how='left', left_on=["CD_CVM", "DATE"], right_on=["CD_CVM", "DT_FIM_EXERC"])
df_history[~df_history["VL_CONTA_ROLLING_YEAR"].isna()]

,CD_CVM,TICKER,DATE,PRICE,DT_INI_EXERC,DT_FIM_EXERC,DS_CONTA,VL_CONTA,EXERC_YEAR,VL_CONTA_ROLLING_YEAR
2109,906,BBDC4,2023-03-31,11.793894,2023-01-01,2023-03-31,Lucro ou Prejuízo,5.473810e+09,2023.0,1.942225e+10
2170,906,BBDC4,2023-06-30,14.969824,2023-04-01,2023-06-30,Lucro ou Prejuízo,4.071265e+09,2023.0,1.628419e+10
4598,1023,BBAS3,2023-03-31,17.840614,2023-01-01,2023-03-31,Lucro ou Prejuízo,7.290982e+09,2023.0,3.216565e+10
4659,1023,BBAS3,2023-06-30,22.752424,2023-04-01,2023-06-30,Lucro ou Prejuízo,9.092514e+09,2023.0,3.253699e+10


In [113]:
df_history = pd.DataFrame()
df_dates = pd.DataFrame({"DATE": pd.date_range(start=df_prices.iloc[0]["DATE"], end=df_prices.iloc[-1]["DATE"])})

for ticker in df_prices["TICKER"].unique():
    df_prices_filtered = df_prices[df_prices["TICKER"] == ticker]

    df_tmp = df_dates.merge(df_prices_filtered, how="left", on="DATE")
    df_tmp = df_tmp.ffill()
    df_history = pd.concat([df_history, df_tmp])
    
    
fundament_type = "Lucro ou Prejuízo"
column_name = "PROFIT"

df_fundaments_filtered = df_fundaments[df_fundaments["DS_CONTA"] == fundament_type]
df_history = df_history.merge(df_fundaments_filtered, how='left', left_on=["CD_CVM", "DATE"], right_on=["CD_CVM", "DT_FIM_EXERC"])
df_history = df_history.drop(["DS_CONTA", "DT_INI_EXERC", "DT_FIM_EXERC", "EXERC_YEAR"], axis=1)
fundament_columns = [column_name, column_name + "_ROLLING_YEAR"]

df_history = df_history.rename(columns={
    "VL_CONTA": fundament_columns[0], 
    "VL_CONTA_ROLLING_YEAR": fundament_columns[1]
    })

df_history.loc[:,fundament_columns] = df_history.groupby("CD_CVM")[fundament_columns].ffill()

df_num_stocks = df_basic_info[["CD_CVM", "NUM_TOTAL"]]
df_history = df_history.merge(df_num_stocks, how='left', on="CD_CVM")

df_history["LPA"] = df_history["PROFIT_ROLLING_YEAR"] / df_history["NUM_TOTAL"]
df_history["PL"] = df_history["PRICE"] / df_history["LPA"]

df_history[~df_history[fundament_columns[1]].isna()]

,DATE,CD_CVM,TICKER,PRICE,PROFIT,PROFIT_ROLLING_YEAR,NUM_TOTAL,LPA,PL
3008,2022-12-31,906.0,BBDC4,12.987669,8.294130e+08,2.121720e+10,10642170228,1.993691,6.514385
3009,2023-01-01,906.0,BBDC4,12.987669,8.294130e+08,2.121720e+10,10642170228,1.993691,6.514385
3010,2023-01-02,906.0,BBDC4,12.644762,8.294130e+08,2.121720e+10,10642170228,1.993691,6.342389
3011,2023-01-03,906.0,BBDC4,12.017268,8.294130e+08,2.121720e+10,10642170228,1.993691,6.027649
3012,2023-01-04,906.0,BBDC4,12.051602,8.294130e+08,2.121720e+10,10642170228,1.993691,6.044871
...,...,...,...,...,...,...,...,...,...
7299,2024-09-30,1023.0,BBAS3,27.180000,9.485229e+09,3.494804e+10,5730834040,6.098246,4.457019
7300,2024-10-01,1023.0,BBAS3,27.080000,9.485229e+09,3.494804e+10,5730834040,6.098246,4.440621
7301,2024-10-02,1023.0,BBAS3,27.150000,9.485229e+09,3.494804e+10,5730834040,6.098246,4.452099
7302,2024-10-03,1023.0,BBAS3,26.900000,9.485229e+09,3.494804e+10,5730834040,6.098246,4.411104
